In [1]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    
    #check if image is grayscale or color
    shape = img.shape
    if(len(shape) > 2):               #its a color image
        mask_color = (255,)*shape[-1]   #shape[-1] = no. at last index
    else:                              #otherwise its a gray image
        mask_color = 255
      
    # Fill the polygon with white
    cv2.fillPoly(mask, vertices, mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [4]:
def hsv_filter(image, min_val_y, max_val_y,  min_val_w, max_val_w):
    """
    A function returning a mask for pixels within min_val - max_val range
    Inputs:
    - image - a BGR image you want to apply function on
    - min_val_y - array of shape (3,) giving minumum HSV values for yellow color
    - max_val_y - array of shape (3,) giving maximum HSV values for yellow color
    - min_val_w - array of shape (3,) giving minumum HSV values for white color
    - max_val_w - array of shape (3,) giving maximum HSV values for white color
    Returns:
    - img_filtered - image of pixels being in given threshold
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask_yellow = cv2.inRange(hsv, min_val_y, max_val_y)
    mask_white = cv2.inRange(hsv, min_val_w, max_val_w)
    mask = cv2.bitwise_or(mask_yellow, mask_white)
    img_filtered = cv2.bitwise_and(image, image, mask=mask)
    
    return img_filtered

In [5]:
def draw_lines(img, lines, color, thickness):
    # If there are no lines to draw, exit.
    if lines is None:
        return
    
    # Create a blank image that matches the original in size.
    line_img = np.zeros_like(img)
    
    # Loop over all lines and draw them on the blank image.
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    
    # Merge the image with the lines onto the original.
    img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)
    
    # Return the modified image.
    return img

In [6]:
def gamma_correction(RGBimage, correct_param = 0.35,equalizeHist = False):
    red = RGBimage[:,:,2]
    green = RGBimage[:,:,1]
    blue = RGBimage[:,:,0]
    
    red = red/255.0
    red = cv2.pow(red, correct_param)
    red = np.uint8(red*255)
    if equalizeHist:
        red = cv2.equalizeHist(red)
    
    green = green/255.0
    green = cv2.pow(green, correct_param)
    green = np.uint8(green*255)
    if equalizeHist:
        green = cv2.equalizeHist(green)
        
    
    blue = blue/255.0
    blue = cv2.pow(blue, correct_param)
    blue = np.uint8(blue*255)
    if equalizeHist:
        blue = cv2.equalizeHist(blue)
    

    output = cv2.merge((blue,green,red))
    return output

In [7]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    
    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1) # <-- Calculating the slope.
            if math.fabs(slope) < 0.2: # <-- Only consider extreme slope
                continue
            if slope <= 0: # <-- If the slope is negative, left group.
                left_line_x.extend([x1, x2])
                left_line_y.extend([y1, y2])
            else: # <-- Otherwise, right group.
                right_line_x.extend([x1, x2])
                right_line_y.extend([y1, y2])

    min_y = img.shape[0] * (3 / 5) # <-- Just below the horizon
    max_y = img.shape[0] # <-- The bottom of the image

    poly_left = np.poly1d(np.polyfit(
        left_line_y,
        left_line_x,
        deg=1
    ))

    left_x_start = int(poly_left(max_y))
    left_x_end = int(poly_left(min_y))

    poly_right = np.poly1d(np.polyfit(
        right_line_y,
        right_line_x,
        deg=1
    ))

    right_x_start = int(poly_right(max_y))
    right_x_end = int(poly_right(min_y))
    
    lines = [[
            [left_x_start, max_y, left_x_end, min_y],
            [right_x_start, max_y, right_x_end, min_y],
            ]]
    
    return lines

In [ ]:
def filter_color(img):
    yellow_min = np.array([65, 80, 80], np.uint8)
    yellow_max = np.array([105, 255, 255], np.uint8)
    yellow_mask = cv2.inRange(img, yellow_min, yellow_max)

    white_min = np.array([0, 0, 200], np.uint8)
    white_max = np.array([255, 80, 255], np.uint8)
    white_mask = cv2.inRange(img, white_min, white_max)
    
    yel-wht-merge = cv2.bitwise_or(yellow_mask, white_mask)
    img = cv2.bitwise_and(img, img, yel-wht-merge)
    return img

In [ ]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [1]:
def pipeline(image):
    """
    An image processing pipeline which will output
    an image with the lane lines annotated.
    """
    original = image
    height, width , ch = image.shape

   #corners
    region_of_interest_points = [
    (0, height),
    (0, height*(3/5)),
    (width, height*(3/5)),
    (width, height),
    ]
    
    #defining color thresholds
    min_val_y = np.array([15,80,190])
    max_val_y = np.array([30,255,255])
    min_val_w = np.array([0,0,180])
    max_val_w = np.array([255, 80, 255])

    # Convert to grayscale here.
    hsv = hsv_filter(cropped, min_val_y, max_val_y,  min_val_w, max_val_w)
    cv2.imshow('hsv', hsv)
    
    # Call Canny Edge Detection here.
    canny = cv2.Canny(gray_image, 150, 250)
    cropped = region_of_interest(img, np.array([region_of_interest_points], np.int32) )
    
    
    
    cv2.imshow('blur', gblur)
    lines = hough_lines(img, 6,np.pi / 60, 160, 20, 25)
    img = draw_lines(original, lines ,thickness=5)
    return img

In [2]:
cap = cv2.VideoCapture('../dataset/day-lane.mp4')
while(cap.isOpened()):
    # Take each frame
    ret, frame = cap.read()
    imageWithLine = pipeline(frame)
    #display image with line over it
    cv2.namedWindow('final', cv2.WINDOW_NORMAL)
    cv2.imshow('final', imageWithLine)
    
    k = cv2.waitKey(25) & 0xFF
    if k == ord('q'):   #stop video
        break
    elif k == ord('s'):  #save frame
        file = '../dataset/dl'+str(i)+'.png'
        cv2.imwrite(file, frame)
        i += 1

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined